In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from dataclasses import dataclass
import io
import json
import os
from pathlib import Path
from pprint import pprint
import re
import requests
import sys
from typing import Optional

if '..' not in sys.path: sys.path.append('..')



# SearchQA dataset exploration
## Setup

In [2]:
DATA_PATH = Path(os.path.expandvars('$HOME')) / 'data'
SEARCHQA_PATH = DATA_PATH / 'SearchQA'
searchqa_train_fpath = SEARCHQA_PATH / 'train.txt'
searchqa_val_fpath = SEARCHQA_PATH / 'val.txt'
searchqa_test_fpath = SEARCHQA_PATH / 'test.txt'

In [12]:
def read_top_lines(fpath, n=5):
    lines = []
    with open(fpath, 'r') as f:
        res = []
        for i in range(n):
            l = f.readline().strip()
            if not l:
                break

            parts = l.split('</s>  <s>')
            n_parts = len(parts)
            parts_new = [f'line {i}:']
            for i, part in enumerate(parts):
                if i < n_parts - 1:
                    part = f'{part}</s>'
                if i > 0:
                    part = f'<s>{part}'
                part = f'    {part}'
                parts_new.append(part)
            l = '\n'.join(parts_new)
            lines.append(l)

    return '\n'.join(lines)


## View samples

In [14]:
s = read_top_lines(searchqa_train_fpath, n=2)
print(s)

line 0:
    <s> assignment 13 nov 30 , 2014 3 letter words 200 question 1 last 8 years life , galileo house arrest espousing man 's theory 2 </s>
    <s> galileo affair wikipedia galileo affair sequence events , beginning around 1610 , culminating trial 1632 galileo , old man , published dialogue concerning two chief galileo kept house arrest death 1642 heliocentrism , theory earth planet , along </s>
    <s> galileo get trouble right , jerk io9 sep 15 , 2011 galileo facing stiff odds published dialogue concerning galileo spending last years life house arrest fair minded , formally granted galileo write theory one espoused aristotelian geocentric view 8 23 17 6k </s>
    <s> web navigation last 8 years life , galileo house arrest espousing man 's theory winter 1971 72 , record 1 , 122 inches snow fell </s>
    <s> costco river cree hours jun 17 , 2013 history last 8 years life , galileo house arrest espousing man 's theory copernicus espn 's top 10 time </s>
    <s> age milton encyclop

In [15]:
s = read_top_lines(searchqa_val_fpath, n=2)
print(s)

line 0:
    <s> summary reviews dangerous summer ernest hemingway commissioned life magazine 1959 write competition great spanish bullfighters high school , hemingway enjoyed working trapeze , school newspaper , wrote first major novel , sun also rises , story american english </s>
    <s> reflections exile essays google books result  </s>
    <s> dangerous summer ernest hemingway reviews dangerous summer 1344 ratings 60 reviews pride matadors , mortal drama fight fight rival matadors try experiences spain 1959 bullfighting season crisscrossed hemingway provides back story , describing visits old haunts , trout </s>
    <s> antonio ordonez dies 66 matador hemingway book dec 21 , 1998 antonio ordonez , leading bullfighter 1950 's , dies age 66 photo married carmen dominguin , sister chief rival , luis miguel spanish ring next summer 1959 , hemingway tow gored ring least eight times 1950 's , go next story </s>
    <s> picasso hemingway dud poem live grenade fascinated stories favorite j